In [1]:
import pandas as pd

In [3]:
corpus = pd.read_csv("Macocu-sl-en-predicted.csv", sep = "\t", index_col = 0)
corpus.head(3)

,biroamer_entities,translation_direction,en_source,en_var_doc,en_var_dom,sl_source,en_domain,sl_domain,average_score,en_doc,sl_doc,en_length,sl_length,punct_ratio,X-GENRE,label_distribution
2584979,No,sl-orig,http://15.liffe.si/?lang_chg=en,B,B,http://15.liffe.si/?lang_chg=sl,15.liffe.si,15.liffe.si,0.936808,It went out with a bang. The evening sparkled ...,Končalo se je razburljivo in z razkošjem. Veče...,574,463,0.103501,Opinion/Argumentation,0.988794
1212933,No,sl-orig,http://16.liffe.si/?lang_chg=en,B,B,http://16.liffe.si/index.php?menu_item=domov,16.liffe.si,16.liffe.si,0.900000,Some days ago the organisers of the 17th Liffe...,Pred dnevi smo se iz 59. mednarodnega filmskeg...,293,184,0.076220,News,0.961600
598330,Yes,sl-orig,http://17.liffe.si/?lang_chg=en,B,B,http://17.liffe.si/?lang_chg=sl,17.liffe.si,17.liffe.si,0.957875,17th LIFFe was brought to an end with the best...,S podelitvijo nagrad in predvajanjem Režiserja...,445,418,0.073930,News,0.997264


In [4]:
corpus["X-GENRE"].value_counts(normalize = True)

Information/Explanation    0.317935
Promotion                  0.308270
News                       0.133635
Instruction                0.106535
Legal                      0.057619
Opinion/Argumentation      0.047767
Other                      0.021551
Forum                      0.003978
Prose/Lyrical              0.002711
Name: X-GENRE, dtype: float64